# GRiTS

In [1]:
# QtCore, needed for interactive visualization, will open a separate python process
from PySide2 import QtCore

%gui qt

In [2]:
import warnings

import fresnel
import fresnel.interact
import freud
import matplotlib.cm
import mbuild as mb
import numpy as np
import PIL

import utils
from cg_compound import CG_Compound

/Users/jenny/miniconda3/envs/grits/lib/python3.7/site-packages/hoomd/meta.py:26: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [41]:
p3ht = mb.load("molfiles/P3HT_16.mol2")
p3ht.translate_to(np.zeros(3))

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    p3ht_mol = p3ht.to_pybel(box = p3ht.boundingbox)
    p3ht_mol.OBMol.PerceiveBondOrders()

In [69]:
cg_beads = [("_B", "c1sccc1"), ("_S", "CCC")]
cg_p3ht = utils.coarse(p3ht_mol, cg_beads)
p3ht_colors = {
    "_B" : 'blue',
    "_S" : 'orange'
}
cg_p3ht.visualize(color_scheme=p3ht_colors, show_atomistic=True).show()
cg_p3ht.visualize(color_scheme=p3ht_colors).show()

/Users/jenny/miniconda3/envs/grits/lib/python3.7/site-packages/mbuild/compound.py:2443: UserWarning: Guessing that "<_B pos=(-1.2320,-2.3766,-1.7036), 0 bonds, id: 5239171664>" is element: "EP"
  atom, element))
/Users/jenny/miniconda3/envs/grits/lib/python3.7/site-packages/mbuild/compound.py:2443: UserWarning: Guessing that "<_S pos=(-0.8683,-2.3467,-1.6166), 0 bonds, id: 5239333520>" is element: "EP"
  atom, element))


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

/Users/jenny/miniconda3/envs/grits/lib/python3.7/site-packages/mbuild/compound.py:2443: UserWarning: Guessing that "<_B pos=(-1.2320,-2.3766,-1.7036), 0 bonds, id: 5240161680>" is element: "EP"
  atom, element))
/Users/jenny/miniconda3/envs/grits/lib/python3.7/site-packages/mbuild/compound.py:2443: UserWarning: Guessing that "<_S pos=(-0.8683,-2.3467,-1.6166), 0 bonds, id: 5239416336>" is element: "EP"
  atom, element))


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [138]:
#def get_particle_i(compound, i):
#    """
#    Get the particle index i
#    
#    Parameters
#    ----------
#    compound : mb.Compound
#    i : int, index of desired particle
#    
#    Returns
#    -------
#    mb.Particle
#    """
#    return [p for p in compound.particles][i]

def get_bonded(compound, particle):
    def is_particle(i,j):
        if i is particle:
            return j
        elif j is particle:
            return i
        else:
            return False
    return [is_particle(i,j) for i,j in compound.bonds() if is_particle(i,j)]

#p0 = [p for p in cg_p3ht.particles()][0]
#print(get_bonded(cg_p3ht, p0))
#print(p0)
p0 = cg_p3ht[0]
#cg_p3ht.get_name_inds(cg_p3ht[0])
def get_index(compound, particle):
    return [p for p in compound.particles()].index(particle)
#get_index(cg_p3ht, p0)
fine_grained.n_particles

496

In [157]:
bead_dict = {
    "_B": {"smiles": "c1sccc1", "ports": [0,2,4], "bonds": {"_B_B": (0,2), "_B_S": (4,0)}},
    "_S": {"smiles": "CCC", "ports": [0,2], "bonds": {"_S_S": (2,0)}}
}

fine_grained = mb.Compound()
bead_lengths = [0]
for bead in cg_p3ht.particles():
    smiles = bead_dict[bead.name]["smiles"]
    b = mb.load(smiles, smiles=True)
    b.translate_to(bead.pos)
    #for port in bead_dict[bead.name]["ports"]:
    #    p = b[port]
    #    b.add(mb.Port(anchor=p), label=str(port))
    fine_grained.add(b)
    bead_lengths.append(fine_grained.n_particles)

cg_copy = CG_Compound.from_mbuild(mb.clone(cg_p3ht))
for i,bead in enumerate(cg_copy.particles()):
    bonds = get_bonded(cg_copy, bead)
    for bond in bonds:
        j = get_index(cg_copy, bond)
        
        names = [bead.name,bond.name]
        bondname = "".join(names)
        try:
            fi,fj = bead_dict[bead.name]["bonds"][bondname]
        except KeyError:
            print("heck")
            bondname = "".join(names[::-1])
            fj,fi = bead_dict[bead.name]["bonds"][bondname]
            
        #print(bead_lengths[i]-1+fi, bead_lengths[j]-1+fj)
        fine_grained.add_bond((fine_grained[bead_lengths[i]+fi], fine_grained[bead_lengths[j]+fj]))
        cg_copy.remove_bond((bead,bond))

#for i,j in cg_p3ht.bonds():
#    print(i,j)
fine_grained.visualize(show_ports=True).show()

print(fine_grained.n_particles)
#cg_copy.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

496


In [158]:
#bead_lengths

In [ ]:
# Fresnel stuff

#scene = utils.draw_scene(cg_p3ht, show_atomistic=True, show_box=True, color="coolwarm")
#view = fresnel.interact.SceneView(scene)
#view.show()
#cam_pos = np.array([1,1,0])*np.array(p3ht.boundingbox.maxs)
#camera = fresnel.camera.Camera()
#camera.position = cam_pos
#camera.up = np.array([0,1,0])
#camera.height = max(p3ht.boundingbox.maxs)
#scene.camera = camera
#fresnel.pathtrace(scene, w=500, h=500, light_samples=10)